In [1]:
#Instalar paquete SPI - SPEI solo instalar una vez
# install.packages("SPEI")
# install.packages("readr")

In [2]:
#Cargar librería SPEI y readr
library(SPEI)
library(readr)


Loading required package: lmomco

Loading required package: parallel

Loading required package: ggplot2

# Package SPEI (1.7) loaded [try SPEINews()].



In [3]:
setwd("../datos/1.0.variables")
getwd()

[1] "C:/Users/Angélica Andrade/Dropbox/documentosA/TFM_UNIR_2022/datos/1.0.variables"

## Funciones para calcular SPI  y SPEI en todas las escalas

In [4]:
spi_escalas <- function(df,escala){
  df1 = data.frame()
  df1 = cbind(df['fecha'])
  spi_es<-spi(df[2:length(df)], scale = escala , distribution = 'Gamma')
  vals=spi_es$fitted
  df1$spi= vals
  return(df1)
}

# Redondear un dataframe, exluyendo los string
redondear <- function(data, digitos) {
  numeros <- vapply(data, is.numeric, FUN.VALUE = logical(1))
  data[,numeros] <- round(data[,numeros], digits = digitos)
  return (data)
}

guardar_csv <- function(data, directorio){
  write.csv(data, file = directorio, row.names = FALSE) # guarda un archivo csv

  df <- read.csv(directorio, header = TRUE, sep = ",", dec = ".") 
  colnames(df) <- c("fecha","100","101","102","105","106","107","108","111","112","113","114","115","116","117","118","119","120","121","200","201","202","203","204","205")
  write.csv(df, file = directorio, row.names = FALSE) # guarda un archivo csv
}


# Calcular el SPEI por dos metodos 1:hargreaves, 2:thornthwaite
# con: - dfstac: datos de las estaciones
#      - escala: escala en meses
#      - dfprec: datos de precipitaciones de todas las estaciones 
#      - dftm: dependiendo del <metodo>, datos de tmax (si metodo:1) y tmed(si metodo:2)
#      - dftmin: datos de tmin de todas las estaciones
#      - metodo: metodo usado para calcular PET

spei_calc <- function(dfstac, escala, dfprec, dftm, dftmin = data.frame(), metodo = 2){
    
    dfspei <- data.frame(cbind(dfprec['fecha']))
    columnas = colnames(dfprec[,2:ncol(dfprec)])

    for (est in 1:24){
        # Asignar los datos de la estacion "est" a una Serie de Tiempo
        tspp <- ts(dfprec[est+1], end=c(2020,12), frequency=12)
        tstm = ts(dftm[est+1], end=c(2020,12), frequency=12)
        
        if (metodo == 1){
            
            tsmn = ts(dftmin[est+1], end=c(2020,12), frequency=12)
            
            # Calcular PET, por el metodo 1:hargreaves
            tspet <- hargreaves(Tmin = tsmn ,Tmax = tstm, lat = dfstac[est,6])
        
        } else {
            
            # Calcular PET, por el metodo 2:thornthwaite
            tspet <- thornthwaite(Tave = tstm, lat = dfstac[est,6])
        }

        # Calcular el balance hídrico: pp - PET
        tsbh = tspp - tspet

        # Calcular el SPEI para una escala "escala"
        tspei = spei(tsbh, scale = escala , distribution = 'log-Logistic')
        
        # Generar el nombre de la columna
        columna <- paste('', substr(columnas[est], 2, 4), sep = '')
        
        # Guardar en los valores obtenidos en dfspei
        fitted_tspei <- tspei$fitted
        dfspei[columna] <- fitted_tspei[,1]
        
    }
    
    return (dfspei)
}

## Definir  escala: 1, 3, 6, 12

In [5]:
escala <- 12

# SPI - Cálculo

In [6]:
# Cargar Dataframe de precipitación de todas las estaciones 
df <- read.csv("chirps_mensual_pp_bc.csv", header = TRUE, sep = ",", dec = ".") 

# Calcular SPI
res = spi_escalas(df,escala)

# Redondear valores y Guardar SPI
nombrespi = paste("../3.0.spi_spei/indices_spi",escala,".csv",sep="")
guardar_csv(redondear(res,3), nombrespi)

# SPEI - Cálculo

In [7]:
# Cargar Dataframes 
precip<-read.csv("chirps_mensual_pp_bc.csv", header = TRUE, sep = ",", dec = ".") 

t_max<-read.csv("nasa_mensual_tmax_bc.csv", header = TRUE, sep = ",", dec = ".")
t_med<-read.csv("nasa_mensual_tmed.csv", header = TRUE, sep = ",", dec = ".")
t_min<-read.csv("nasa_mensual_tmin_bc.csv", header = TRUE, sep = ",", dec = ".") 

estacion<-read.csv("../1.0.Estaciones/estaciones_senamhi_generado.csv", header = TRUE, sep = ",", dec = ".") 

# para metodo 1 = hargreaves
datospei2 = spei_calc(estacion, escala, precip, t_max, t_min, 1)

# para metodo 2 = thornthwaite
#datospei2 = spei_calc(estacion, escala, precip, t_med)

# Redondear valores y Guardar SPEI
nombrespei = paste("../3.0.spi_spei/indices_spei",escala,".csv",sep="")
write.csv(redondear(datospei2,3), file = nombrespei, row.names = FALSE)
